# Autocorrelation: Exploring Rossmann Drug Store Sales Data
### Jonathan Balaban

This lab sets the foundation for ARIMA modeling. We will get to know our dataset, cover basic theory, and explore Python methods for managing time series data.

## Objectives:

- Explore Rossmann data
- Introduce technical concepts related to time series and ARIMA modeling 
- Calculate moving averages
- Analyze Autocorrelations

## Context:

> Key Definition: Time series data is any data that is captured across time.

We have captured daily sales data for [Rossmann](https://en.wikipedia.org/wiki/Rossmann_(company)) (German mart/pharmacy, like CVS). If we can use past sales data to predict future sales, we have autocorrelation among the feature "Sales". This is a solid foundation for modeling and we can determine how far back our model should look. The past two days might be insightful, but if we're predicting Monday sales, so might the previous two Mondays, etc.

However, if we go back too far, we complicate our model, and as expected, long ago data will not be very correlated with current sales.

Time series data usually contains more than meets the eye, and can often be decomposed into trend, seasonal, and random fluctuation components.

![Decomposition](http://rstatistics.net/wp-content/uploads/2014/09/Multiplicative-Decomposition-of-Time-series.png)

- Trends
    - Up
    - Down
    - Flat
    - Larger trends can be made up of smaller trends
    - There is no defined timeframe for what constitutes a trend; it depends on your goals
- Seasonal Effects
    - Weekend retail sales spikes
    - Holiday shopping
    - Energy requirement changes with annual weather patterns
    - Note: twitter spikes when news happens are not seasonal; they aren't regular and predictable
- Random Fluctuations
    - The human element
    - Aggregations of small influencers
    - Observation errors
    - The smaller this is in relation to Trend and Seasonal, the better we can predict the future
    
Time series models fall into [two camps](http://www.abs.gov.au/websitedbs/D3310114.nsf/home/Time+Series+Analysis:+The+Basics#HOW%20DO%20I%20KNOW%20WHICH%20DECOMPOSITION):
- Additive
    - Data = Trend + Seasonal + Random
    - What we will be using for our modeling
- Multiplicative
    - Data = Trend x Seasonal x Random
    - As easy to fit as Additive if we take the log
    - log(Data) = log(Trend x Seasonal x Random)

We should use multiplicative models when the percentage change of our data is more important than the absolute value change (e.g. stocks, commodities); as the trend rises and our values grow, we see amplitude growth in seasonal and random fluctuations.

## Time Series Modeling Process
Time series model selection is driven by the Trend and Seasonal components of our raw data. The general approach for analysis looks like this:

1. Plot the data and determine Trends and Seasonality
    1. Difference the data (multiple times if needed) to remove trends for [certain model applications](https://en.wikipedia.org/wiki/Stationary_process)
    1. Stationairity is needed for ARMA models
1. Determine if we have additive or multiplicative data patterns
1. Select the appropriate algorithm based on the chart below
1. Determine if model selection is correct with these tools
    - Ljung-Box Test
    - Residual Errors (Normal Distribution with zero mean and constant variance-homoskedastic)
    - Autocorrelation Function (ACF)
    - Partial Autocorrelation Function (PACF)

Algorithm | Trend | Seasonal | Correlations
---|---|---|---
ARIMA | X |X|X
SMA Smoothing |X||
Simple Exponential Smoothing |X||
Seasonal Adjustment |X|X|
Holt's Exponential Smoothing |X||
Holt-Winters |X|X|




In [ ]:
# import packages and data
import pandas as pd, numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

data = pd.read_csv('data/rossmann.csv', skipinitialspace=True, low_memory=False)

In [ ]:
# we are most interested in `Date` column that contains date of sales per store; convert to `DateTime` and set as index
# pull year and month as features
data['Date'] = pd.to_datetime(data['Date'])
data.set_index('Date', inplace=True)

data['Year'] = data.index.year
data['Month'] = data.index.month

In [ ]:
# sort dates to ascending and view
data.sort_index(inplace=True)
data.head()
# notice Monday is DayOfWeek 1

In [ ]:
# check index for missing days

# first find the number of unique indices
len(data.index.unique())

In [ ]:
# then calculate the delta in time plus one (inclusive of first day)
data.index.max() - data.index.min()

# counting the first day, we have a match: no missing dates!

In [ ]:
# describe and EDA
data.describe().round(2)

In [ ]:
# subset data to open dates
df = data[data.Open==1]

In [ ]:
# pivot table of average sales with rows=month and cols as promotion or not
pt = df.pivot_table(index='Month', columns='Promo', values='Sales')
pt

In [ ]:
# plot average sales by month and promo
pt.plot();

In [ ]:
# plot average customers by month and promo
df.pivot_table(index='Month', columns='Promo', values='Customers').plot();

In [ ]:
# df of store 1 open day sales
store1 = df[df.Store == 1]

To compare sales on holidays, we can compare the sales using box-plots, which allows us to compare the distribution of sales on holidays against all other days. On state holidays the store is closed (and as a nice sanity check there are 0 sales), and on school holidays the sales are relatively similar.

In [ ]:
# do school holidays affect sales?
sns.catplot(
    x='SchoolHoliday',
    y='Sales',
    data=store1,
    kind='box');

In [ ]:
# does day of week affect sales?
sns.catplot(
    hue='SchoolHoliday',
    x='DayOfWeek',
    y='Sales',
    data=store1,
    kind='box',
    height=8);
# notice closed on Sundays

Lastly, we want to identify larger-scale trends in our data. How did sales change from 2014 to 2015? Were there any particularly interesting outliers in terms of sales or customer visits?

In [ ]:
# plot store 1 sales when open
store1.Sales.plot(figsize=(18,7));

In [ ]:
# plot store 1 customer count when open
store1.Customers.plot(figsize=(18,7));

## Autocorrelation

To measure how much the sales are correlated with each other, we want to compute the _autocorrelation_ of the 'Sales' column. In pandas, we do this we with the `autocorr` function.

`autocorr` takes one argument, the `lag` - which is how many prior data points should be used to compute the correlation. If we set the `lag` to 1, we compute the correlation between every point and the point directly preceding it, while setting `lag` to 10, computes the correlation between every point and the point 10 days earlier.

In [ ]:
# resample all store data to average daily sales
daily_average_sales = df.Sales.resample('D').mean()

daily_average_sales.head(10)

In [ ]:
# check autocorrelation for previous two weeks
for i in range(1,16):
    print(i, daily_average_sales.autocorr(lag=i))

In [ ]:
# check how many days after counts as a year, since some days are closed
len(data.index.unique()) - len(daily_average_sales.index.unique())

# across all stores, some are always open

In [ ]:
# check yearly autocorr
for i in range(360,371):
    print(i, daily_average_sales.autocorr(lag=i))
    
# note the spike around 364-365

In [ ]:
# plot autocorrelation for different lags using pandas
from pandas.plotting import autocorrelation_plot
plt.figure(figsize=(20,7))

autocorrelation_plot(daily_average_sales)
plt.xlim(0,35);

## Moving/Rolling Averages

If we want to investigate trends over time in sales, as always, we will start by computing simple aggregates.  We want to know what the mean and median sales were for each month and year.

In Pandas, this is performed using the `resample` command, which is very similar to the `groupby` command. It allows us to group over different time intervals.

We can use `data.resample` and provide as arguments:
    - The level on which to roll-up to, 'D' for day, 'W' for week, 'M' for month, 'A' for year
    - What aggregation to perform: 'mean', 'median', 'sum', etc.

In [ ]:
# resample original sales data total by week
weekly_average_sales = df.Sales.resample('W').mean()

weekly_average_sales.plot(figsize=(20,5));

While identifying the weekly averages are useful, we often want to compare the sales data of a date to a smaller window. To understand holidays sales, we don't want to compare late December with the entire month, but perhaps a few days surrounding it. We can do this using rolling averages.

In [ ]:
# find rolling daily mean

daily_average_sales.rolling(window=3, center=True).mean().plot(figsize=(20,7));

# this gives us a bit of smoothing to exclude extreme events

`rolling` has these important parameters:
- the first is the series to aggregate
- `window` is the number of days to include in the average
- `center` is whether the window should be centered on the date or use data prior to that date
- `freq` level to roll-up averages to (as in `resample`). `D` for day, `M` for month or `A` for year, etc.

Instead of plotting the full timeseries, we can plot the rolling mean instead, which smooths random changes in sales as well as removing outliers, helping us identify larger trends.

In [ ]:
# plot 5-week centered rolling mean

weekly_average_sales.rolling(window=5, center=True).mean().plot(figsize=(18,6));

## Pandas Window functions
Pandas `rolling` is an example of Pandas window function capabilities. Window functions operate on a set of N consecutive rows (a window) and produce an output: mean, median, min, max, sum, etc.

Another common one is `diff`, which takes the difference over time. `pd.diff` takes one arugment, `periods`, which is how many prior rows to use for the difference. This is *extremely* useful for getting our data stationary!

In addition to `rolling` functions, Pandas provides a similar collection of `expanding` functions, which instead of a window, use all values up until that time.

In [ ]:
# calculate diff for open store 1 data
store1['Diff'] = store1['Sales'].diff(periods=1)
store1.Diff.plot(figsize=(20,6));

In [ ]:
# plot 30-day rolling mean
store1.Sales.rolling(30).mean().plot(figsize=(20,6));
# notice the smoothing

In [ ]:
# compute average daily expanding sales
daily_average_sales.expanding(min_periods=1).mean().head()

In [ ]:
# does expanding sales at the last row work as assumed?
print(daily_average_sales.expanding(min_periods=1).mean().iloc[-1])
print
print(daily_average_sales.mean())

# yes, minus rounding errors!

# Exercises

In [ ]:
# plot the distribution of sales by month and compare the effect of promotions
sns.catplot(
    col='Open',
    hue='Promo',
    x='Month',
    y='Sales',
    data=store1, 
    kind='box');

In [ ]:
# Are sales more correlated with the prior day, day of week, last month, or last year?

# remake "daily_average_sales" but include Open data
average_daily_sales = df[['Sales', 'Open']].resample('D').mean()

print('Correlation with last day: {}'.format(average_daily_sales['Sales'].autocorr(lag=1)))
print('Correlation with last week: {}'.format(average_daily_sales['Sales'].autocorr(lag=7)))
print('Correlation with last month: {}'.format(average_daily_sales['Sales'].autocorr(lag=30)))
print('Correlation with last year: {}'.format(average_daily_sales['Sales'].autocorr(lag=365)))

In [ ]:
# plot the 15 day rolling mean of customers in the stores
average_daily_sales.Sales.rolling(window=15).mean().plot(figsize=(18,6));

In [ ]:
# identify the date with largest drop in average sales from previous cycles: daily, weekly, etc.
total_daily = df[['Sales', 'Open']].resample('D').sum()
total_daily['Diff'] = total_daily.Sales.diff(periods=1)

total_daily.sort_values(by='Diff').head()

In [ ]:
# compute the total sales up until Dec. 2014
total_daily_sales = df.Sales.resample('D').sum()
total_daily_sales.expanding().sum()['2014-12'].head()

In [ ]:
# When were the largest differences between 15-day moving/rolling averages?
x = total_daily_sales.rolling(window=15).mean().diff(1)

In [ ]:
# sort values
x.sort_values(ascending=True).head(10)

# Unsurprisingly, they occur at the beginning of every year after the holiday season

### How Differencing Works

Let's see how we can standardize our data (remove trends and changes in variance) via different levels of differencing. This is important, as standardized data is a requirement for ARIMA modeling.

In [ ]:
# create a play dataframe from 1-10, regular and squared to test differencing works
play = pd.DataFrame([[x for x in range(1,11)], [x**2 for x in range(1,11)]]).T
play.columns = ['original', 'squared']
play

In [ ]:
# take diffs of our two series until they're stationary (mean doesn't change for sub-windows)
play.original.diff()
# this is similar to taking a first-order derivative

In [ ]:
play.squared.diff().diff()
# notice we need to difference twice on an exponential trend, and every time we do, we lose a bit of data

## Objectives:

- Explore Rossmann data
- Introduce technical concepts related to time series and ARIMA modeling 
- Calculate moving averages
- Analyze Autocorrelations